In [1]:
import numpy as np

# 다른 온도 값을 사용하여 확률 분포의 가중치 바꾸기
# original_distribution 은 전체 합이 1인 1D 넘파이 배열
# temperature는 출력 분포의 엔트로피 양을 결정
def reweight_distribution(original_distribution, temperature=0.5):
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    # 원본 분포의 가중치를 변경하며 반환. 
    # 이 분포의 합은 1이 아닐 수 있으므로 새로운 분포의 합으로 나눈다.
    return distribution / np.sum(distribution) 


In [2]:
# 글자 수준의 LSTM 텍스트 생성 모델 구현

#데이터 전처리
import keras
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('말뭉치 크기:', len(text))

Using TensorFlow backend.


606208/600901 [==============================] - 1s 2us/step
말뭉치 크기: 600893


In [19]:
# 글자 시퀀스 벡터화
maxlen = 60 # 60개의 글자로 된 시퀀스를 추출
step = 3 # 세 글자씩 건너뛰면서 새로운 시퀀스를 샘플링

sentences = [] # 추출한 시퀀스
next_chars = [] # 타깃(시퀀스 다음 글자)

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
    
print('시퀀스 개수:', len(sentences))

chars = sorted(list(set(text))) # 말뭉치에서 고유한 글자 리스트
print('고유한 글자:', len(chars))

char_indices = dict((char, chars.index(char)) for char in chars) # chars 리스트에 있는 글자와 글자의 인덱스를 매핑

print('벡터화...')
# 글자를 원-핫 인코딩하여 0과 1의 이진 배열로 바꾼다.
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


시퀀스 개수: 200278
고유한 글자: 57
벡터화...


In [23]:
y[0]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False])

In [14]:
print(len(sentences))

60


In [13]:
print(next_chars)

['r', 'n', ' ', 'o', 'd', 'o', 's', 'p', 't', 'g', 'h', ' ', 'l', 'h', 'o', 'p', 'r', ' ', ' ', ' ', 'r', 's', 'h', ' ', 'v', 'b', 'n', 'o', 'a', 's', ',', 'a', ' ', 'i', 'd', 'o', 'n', 'r', 'a', ' ', 'm', '-', 'h', ' ', 'e', 'e', 'i', 'e', 'e', 'o', 'n', 's', 'n', 'c', 'm', ' ', 'p', 't', 'i', ' ', 't', 'w', 'c', 't', 'y', 'a', ' ', 'u', 'l', 'p', 'd', 'h', 'r', 'd', 'e', 'e', 't', 't', 't', ' ', 'v', 'b', 'n', 'n', 'i', 'e', 'a', ' ', 's', 'm', ' ', 't', 'd', 'f', '\n', 'n', 'n', 'a', 'o', 'n', 'c', 't', 'n', ' ', 'e', 'a', 'n', 'e', 'a', 'o', 'd', 'e', 'e', ' ', ' ', ' ', 'n', 'a', '\n', ' ', 'e', 'n', 'e', 'r', 'k', 'd', 'f', 'o', 'a', 't', 'd', 'w', 'h', 'a', 'a', ' ', 's', 'u', 'g', ' ', 'e', '-', ',', 'n', 'e', ' ', ' ', 'a', 's', 't', 'l', ' ', 'r', 'h', 'e', 'r', 's', 'f', 'r', 'w', ' ', 'i', 'a', ' ', 'a', 'i', 'h', ' ', 'l', 'n', 't', 't', 'l', 'd', 'm', 'l', 's', 'n', 'h', 'g', 'u', '-', 'a', 'm', 'e', 't', 't', 't', 's', 't', 't', 'l', 't', 'a', '.', 'u', 't', 's', 'a', 's

In [24]:
# 네트워크 구성
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [25]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [26]:
# 모델의 예측이 주어졌을 때 새로운 글자를 샘플링하는 함수
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [28]:
# 텍스트 생성 루프
import random
import sys

random.seed(42)
start_index = random.randint(0, len(text) - maxlen - 1)

for epoch in range(1, 60): # 60 에포크 동안 모델을 훈련
    print('에포크', epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    
    seed_text = text[start_index: start_index + maxlen] # 무작위로 시드 텍스트를 선택
    print('--- 시드 텍스트: "' + seed_text + '"')
    
    for temperature in [0.2, 0.5, 1.0, 1.2]: # 여러가지 샘플링 온도 시도
        print('---- 온도: ', temperature)
        generated_text = seed_text
        sys.stdout.write(generated_text)
        
        for i in range(400): # 시드 텍스트에서 시작해서 400개의 글자를 생성한다
            # 지금까지 생성한 글자를 원-핫 인코딩으로 바꾼다
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
                
            # 다음 글자를 샘플링한다
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            
            generated_text += next_char
            generated_text = generated_text[1:]
            
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

에포크 1
Epoch 1/1
200278/200278 [==============================] - 222s 1ms/step - loss: 1.8861
--- 시드 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
---- 온도:  0.2
the slowly ascending ranks and classes, in which,
through for the sense of the sense of the says the existions and not the say, and the suppicience of the man and some of the sense the self-conservent of the same and the present of the man and some of the strenged the man and the disconcess and the same of the conserves the greater the more spirition of the more some of the man and the present of the present of the same of the says the says the say, the se
---- 온도:  0.5
the slowly ascending ranks and classes, in which,
through for the ofe science are the one of the cale and disconcessibles the exisint of the present and the eneselly discordinged and of the conserves the generation, there double instinction the existions and fallives of men that the rearady a concence of the and sould to the conserving of t

KeyboardInterrupt: 